In [10]:
import os
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras import layers, models, Sequential
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, MaxPooling2D, Dropout, Flatten, Dense, Conv1D, MaxPooling1D
)

In [11]:
def load_dataset(base_path, mode="all"):
    """
    Load dataset from the specified base_path.
    Modes:
        - "all": Load all columns (MFCC + Delta + Delta-Delta)
        - "mfcc": Load first third (MFCC only)
        - "mfcc_delta": Load first two-thirds (MFCC + Delta)
    """
    X, y = [], []
    labels = sorted(os.listdir(base_path))

    for label in labels:
        label_dir = os.path.join(base_path, label)
        if not os.path.isdir(label_dir):
            continue
        for file in os.listdir(label_dir):
            if file.endswith(".csv"):
                csv_path = os.path.join(label_dir, file)
                data = pd.read_csv(csv_path).values
                if mode == "all":
                    features = data
                elif mode == "mfcc":
                    features = data[:, :data.shape[1] // 3]
                elif mode == "mfcc_delta":
                    features = data[:, :2 * data.shape[1] // 3]
                else:
                    raise ValueError("Invalid mode specified.")
                X.append(features)
                y.append(labels.index(label))
    return np.array(X, dtype=object), np.array(y)


In [12]:
def prepare_2d_data(X):
    """
    Prepare data for 2D CNN input (add channel dimension).
    """
    max_timesteps = max(x.shape[0] for x in X)
    feature_dim = X[0].shape[1]
    X_padded = np.zeros((len(X), max_timesteps, feature_dim))
    for i, x in enumerate(X):
        X_padded[i, :x.shape[0], :] = x
    return X_padded[..., np.newaxis]


In [13]:
def train_cnn_2d(X_train, y_train, input_shape, classes_num=6):
    """
    Train the complex CNN model on 2D input data.
    """
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(input_shape[0], input_shape[1], 1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(classes_num, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    return model


In [14]:
def train_svm(X_train, y_train):
    """
    Train an SVM on column-wise mean features.
    """
    X_train_mean = np.array([x.mean(axis=0) for x in X_train])
    svm = SVC(kernel='linear')
    svm.fit(X_train_mean, y_train)
    return svm


In [15]:
def train_cnn_1d(X_train, y_train, input_shape, classes_num=6):
    """
    Train the 1D CNN model.
    """
    model = models.Sequential()
    model.add(layers.Conv1D(64, 3, activation='relu', input_shape=input_shape))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Conv1D(128, 3, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Conv1D(128, 3, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(classes_num, activation='softmax'))

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)
    return model

In [16]:
def evaluate_model(model, X_test, y_test):
    """
    Evaluate a given model (CNN) on test data.
    """
    _, accuracy = model.evaluate(X_test, y_test, verbose=0)
    return accuracy

In [17]:
def evaluate_svm(svm, X_test, y_test):
    """
    Evaluate SVM on test data.
    """
    X_test_mean = np.array([x.mean(axis=0) for x in X_test])
    y_pred = svm.predict(X_test_mean)
    return accuracy_score(y_test, y_pred)


In [18]:
# Dataset paths
train_path = "../Datasets_mfcc_22/Environment 1/Training"
test_env1_path = "../Datasets_mfcc_22/Environment 1/Testing"
test_env2_path = "../Datasets_mfcc_22/Environment 2"

# Load datasets
X_train, y_train = load_dataset(train_path, mode="all")
X_test_env1, y_test_env1 = load_dataset(test_env1_path, mode="all")
X_test_env2, y_test_env2 = load_dataset(test_env2_path, mode="all")
print(X_train.shape)
# CNN 2D
input_shape_2d = (max(x.shape[0] for x in X_train), X_train[0].shape[1])
X_train_2d = prepare_2d_data(X_train)
X_test_env1_2d = prepare_2d_data(X_test_env1)
X_test_env2_2d = prepare_2d_data(X_test_env2)

cnn_2d_model = train_cnn_2d(X_train_2d, y_train, input_shape_2d)
acc_env1_cnn2d = evaluate_model(cnn_2d_model, X_test_env1_2d, y_test_env1)
acc_env2_cnn2d = evaluate_model(cnn_2d_model, X_test_env2_2d, y_test_env2)

# SVM
svm_model = train_svm(X_train, y_train)
acc_env1_svm = evaluate_svm(svm_model, X_test_env1, y_test_env1)
acc_env2_svm = evaluate_svm(svm_model, X_test_env2, y_test_env2)

# CNN 1D
input_shape_1d = (X_train[0].shape[1], 1)
X_train_1d = np.array([x.mean(axis=0)[:, np.newaxis] for x in X_train], dtype=np.float32)
X_test_env1_1d = np.array([x.mean(axis=0)[:, np.newaxis] for x in X_test_env1], dtype=np.float32)
X_test_env2_1d = np.array([x.mean(axis=0)[:, np.newaxis] for x in X_test_env2], dtype=np.float32)

# X_train_1d = np.array(X_train_1d, dtype=np.float32)  
y_train = np.array(y_train, dtype=np.int32)          
cnn_1d_model = train_cnn_1d(X_train_1d, y_train, input_shape_1d)
acc_env1_cnn1d = evaluate_model(cnn_1d_model, X_test_env1_1d, y_test_env1)
acc_env2_cnn1d = evaluate_model(cnn_1d_model, X_test_env2_1d, y_test_env2)

# Display results
print("Results Comparison:")
print(f"2D CNN - Environment 1: {acc_env1_cnn2d:.4f}, Environment 2: {acc_env2_cnn2d:.4f}")
print(f"SVM - Environment 1: {acc_env1_svm:.4f}, Environment 2: {acc_env2_svm:.4f}")
print(f"1D CNN - Environment 1: {acc_env1_cnn1d:.4f}, Environment 2: {acc_env2_cnn1d:.4f}")

(180, 130, 66)
Epoch 1/10
6/6 [==============================] - 3s 175ms/step - loss: 2.6634 - accuracy: 0.2222
Epoch 2/10
6/6 [==============================] - 1s 164ms/step - loss: 1.6711 - accuracy: 0.4778
Epoch 3/10
6/6 [==============================] - 1s 157ms/step - loss: 1.0089 - accuracy: 0.5778
Epoch 4/10
6/6 [==============================] - 1s 160ms/step - loss: 0.6288 - accuracy: 0.7833
Epoch 5/10
6/6 [==============================] - 1s 169ms/step - loss: 0.5200 - accuracy: 0.8167
Epoch 6/10
6/6 [==============================] - 1s 174ms/step - loss: 0.3502 - accuracy: 0.8889
Epoch 7/10
6/6 [==============================] - 1s 164ms/step - loss: 0.1740 - accuracy: 0.9556
Epoch 8/10
6/6 [==============================] - 1s 161ms/step - loss: 0.1743 - accuracy: 0.9611
Epoch 9/10
6/6 [==============================] - 1s 166ms/step - loss: 0.1413 - accuracy: 0.9722
Epoch 10/10
6/6 [==============================] - 1s 160ms/step - loss: 0.1481 - accuracy: 0.9722
Epoc